# PYNQ Car using DFROBOT Turtle: 2WD Mobile Robot Platform

![](images/pynq_car.jpg)
---

## Aim

* This notebook illustrates how to use available APIs for the Geared Motor and DC Motor Driver on PYNQ-Z2 PMOD interface and build the PYNQ Car using DFROBOT Turtle: 2WD Mobile Robot Platform.

## References
* [TT Geared Motor with Back Shaft](https://www.dfrobot.com/product-100.html) 
* [2x1.2A DC Motor Driver](https://www.dfrobot.com/product-1705.html)
* [Turtle: 2WD Mobile Robot Platform](https://www.dfrobot.com/product-65.html) 
* [Grove Base Shield V2.0](https://www.seeedstudio.com/Base-Shield-V2.html)   
* [Grove Ultrasonic Ranger](https://www.seeedstudio.com/Grove-Ultrasonic-Distance-Sensor.html)   
* [Grove Line Finder](https://www.seeedstudio.com/Grove-Line-Finder-v1-1.html)    
* [Grove Relay](https://www.seeedstudio.com/Grove-Relay.html)  

## Last revised
* 21 June 2021
    + Initial version
---

## Load _base_ Overlay

In [1]:
from pynq.overlays.base import BaseOverlay
from pynq_peripherals import PmodGroveAdapter, ArduinoSEEEDGroveAdapter
from dfrobot import PynqCar

base = BaseOverlay('base.bit')

## Control the motor with PYNQ PMOD

<div class="alert alert-box alert-warning"><ul>
    <h4 class="alert-heading">Make Physical Connections </h4>
    <li>Connect the motors to motor driver and the motor driver should be powered by a DC source larger than 6V. </li><li>Connect the green wires from motor driver PWM1 and PWM2 to PMODA pin 2 and PMODB pin 2 respectively, connect the green wire of DIR1 and DIR2 to PMODA pin 6 and PMODB pin 6 repectively. </li><li> The black wires from motor driver should connect to PMOD ground pins.</li><li> The red wires from motor driver are left floating, hence they can be unplugged or cut from the connector.</li>
</ul>
</div>

### Adapter configuration

In [2]:
PMODA_adapter = PmodGroveAdapter(base.PMODA, G4='geared_motor')
PMODB_adapter = PmodGroveAdapter(base.PMODB, G4='geared_motor')
ARDUINO_adapter = ArduinoSEEEDGroveAdapter(base.ARDUINO, D5='grove_line_finder', D6='grove_line_finder', D7='grove_usranger', D8='grove_relay')

### Define device object

In [3]:
motor1 = PMODA_adapter.G4
motor2 = PMODB_adapter.G4
line_finder1 = ARDUINO_adapter.D5
line_finder2 = ARDUINO_adapter.D6
usranger = ARDUINO_adapter.D7
motor_sw = ARDUINO_adapter.D8

In [13]:
# switch on motor driver
motor_sw.on()

In [5]:
car = PynqCar(motor1, motor2)

#### Execute below cell to turn OFF motor driver
This will save the battery life when car is not in use

In [12]:
# switch off motor driver
motor_sw.off()

<div class="alert alert-box alert-info"><ul>
    <h4 class="alert-heading">Notes </h4>
    <li><b>Set the speed:</b> Valid values are from -100 to 100. The car will go forward if the speed is positive and go backword if the speed is negative  </li><li><b>Steering:</b> Valid values are from -100 to 100</li><li> <b>Set the speed offset:</b> The speed offset is used to offset the speed difference of the two motors to make sure the car goes straight with 0 steering</li>
</ul>
</div>

<div class="alert alert-box alert-warning"><ul>
    <h4 class="alert-heading">Make Physical Connections </h4>
    <li>Insert the SEEED Grove Base Shield into the Arduino connector on the board. Connect two Grove Line Finder modules to D5 and D6 connector of the Grove Base Shield respectively.</li>
    <li>Connect the Grove Usranger module to D7 connector of the Grove Base Shield</li>
    <li>Connect the Grove Relay module to D8 connector of the Grove Base Shield</li>
    <li>The two line finders should be aligned together in front of the turtle.</li>
</ul>
</div>

## Line tracking
Two line finders are required

In [7]:
car.speed_offset = 0
car.speed_diff = 0
car.set_speed(20)
while True:
    try:
        if not line_finder1.line_found() and  not line_finder2.line_found():
            car.steering(0)
        elif line_finder1.line_found() and not line_finder2.line_found():
            car.steering(20)
        elif not line_finder1.line_found() and line_finder2.line_found():
            car.steering(-20)
        else:
            car.stop()
    except KeyboardInterrupt:
        car.stop()
        break

---

## Barrier avoidance
* Detect barrier and stop

In [ ]:
import time

car.speed_offset = 0
car.speed_diff = 0
car.set_speed(40)
while True: 
    try:
        if usranger.get_distance() < 20:
            car.stop()
            car.steering(40)
            car.speed_offset = 0
            car.speed_diff = 0
            car.set_speed(40)
        else:
            car.set_speed(40)
    except KeyboardInterrupt:
        car.stop()
        break

---

## Using line finder to score if the car goes stright

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from time import sleep

In [ ]:
def control(speed, steering):
    global n_error, n_fit
    car.set_speed(speed)
    car.steering(steering)
    if line_finder1.line_found() and line_finder2.line_found():
        n_fit += 1
    else:
        n_error += 1
    print('score: %f' % (n_fit/(n_fit+n_error)))

### Run the sell below and control the turtle with the widget

In [ ]:
n_error = 0
n_fit = 0
interact(control, speed=(-100, 100, 1), steering=(-50, 50, 1));

Copyright (C) 2021 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause

----

----